In [1]:
import pandas as pd

In [2]:
df_projects = (
    pd
    .read_csv('data/op_rf4_contracts_by_application.csv')
    .groupby(['application_id', 'project_name', 'oso_project_filename', 'contract_address'])['network']
    .agg(lambda x: ','.join(list(x)))
    .reset_index()
    .set_index('contract_address')
)
df_projects.tail(1)

,application_id,project_name,oso_project_filename,network
contract_address,,,,
0xa61550e9ddd2797e16489db09343162be98d9483,0xffffeb1d3576c1e847701c225975ee1a10259e05366b...,Rainbow (Rainbow Router),rainbow,ZORA


In [3]:
base = pd.read_csv('data/os_audit/base.csv', skiprows=1)
base['verified_network'] = 'BASE'
op = pd.read_csv('data/os_audit/optimism.csv', skiprows=1)
op['verified_network'] = 'OPTIMISM'
eth = pd.read_csv('data/os_audit/mainnet.csv', skiprows=1)
eth['verified_network'] = 'MAINNET'
contracts = pd.concat([base, op, eth], axis=0)
contracts['contract_address'] = contracts['ContractAddress'].str.lower()

df_contracts = (
    contracts
    .groupby('contract_address')
    .agg({
        'verified_network': lambda x: ','.join(list(x)),
        'ContractName': lambda x: ','.join(set(x)),
    })
)

df_contracts.tail(1)

,verified_network,ContractName
contract_address,,
0xfff80af12a3cf11d444170b86d5498e5583f9da2,OPTIMISM,ArgusRootAuthorizer


In [4]:
df_activity = (
    pd
    .read_parquet('data/op_rf4_contracts_by_project.parquet')
    .groupby('contract_address')
    .agg({'total_gas': 'sum'})
    .reset_index()
    .set_index('contract_address')
)
df_activity.tail(1)

,total_gas
contract_address,
0xffffffaeff0b96ea8e4f94b2253f31abdd875847,0.003331


In [5]:
df_repos = pd.read_csv('data/os_audit/repo_stats.csv')
df_repos = df_repos[df_repos['has_contracts'] == True]
df_repos = df_repos.groupby('application_id').agg({
    'fork_count': 'sum',
    'star_count': 'sum',
    'repo_older_than_1_month': lambda x: all(x),
    'license_check': lambda x: all(x=='Permissive'),
    'repo_activity_check': lambda x: all(x=='OK')
})
df_repos.tail(1)

,fork_count,star_count,repo_older_than_1_month,license_check,repo_activity_check
application_id,,,,,
0xfdf7761b1307fc08a682fcacdaa8f5c46c64dce2fea74ea4ecf30297f7690a54,0.0,0.0,True,False,False


In [6]:
df_audit = pd.read_csv('data/os_audit/bix_review.csv')
df_audit = df_audit[df_audit['Issue With OS Tag'] == 'X']
df_audit = df_audit[['application_id', 'Bix Comment', 'Contract with issue']]
df_audit

,application_id,Bix Comment,Contract with issue
0,0xd61084147c0e08c9a304348584c6bd0e9974a4073bff...,Contracts submit were LGPL-3.0-only. Github do...,https://github.com/zeriontech/defi-sdk
1,0x935315db8c17993464cfb28337442ad60c6cd4343051...,Both contracts that were submit to the round w...,https://basescan.org/address/0xe3D61290740B8a0...
2,0x526522707f6c225a9bc7ea54e572f995eb3c794f3475...,Contracts submit are a mix of MIT and NO LICEN...,https://optimistic.etherscan.io/address/0x9761...
3,0x41b2ae8a2ead458ff7c85549a9a11380353e1ae4b234...,Contracts submit were GPL-3.0 and BUSL-1.1. Gi...,https://basescan.org/address/0xc35DADB65012eC5...
4,0xf9a4e3b240bf6d40ee533e5ed63a8288b1a739ee382b...,Contracts submit are AGPL-3.0-or-later or UNLI...,https://optimistic.etherscan.io/address/0xaaf6...
5,0x848488156d9bfc5356583da57253193a258acebcdbc3...,Contract submit is unverified. Github license ...,https://basescan.org/address/0xD0aFE09956edC99...
6,0xf1d424db61a36ef7e1bef62f6c62f00b2e5ff74ce12c...,Contract submit are MIT and UNLICENSED. Github...,https://basescan.org/address/0x6A89b05f2c0Ce13...
7,0x79fa54d6c030c7803827e96933fc93b092f638079c73...,Contracts submit does not have a license. Gith...,https://github.com/SmolDapp/coreContracts/blob...
8,0x7539f43dc91f9c406c83a06339708e4033cdbf630916...,"Contract submit were BUSL-1.1, AGPL-3.0, MIT, ...",https://basescan.org/address/0x36291d2D51a0122...
9,0xffffeb1d3576c1e847701c225975ee1a10259e05366b...,Contracts submit are GPL-3.0 and unverified. G...,https://explorer.zora.energy/address/0xA61550E...


In [7]:
df = (
    df_projects
    .join(df_contracts)
    .join(df_activity)
    .reset_index()
     .merge(df_repos, on='application_id')
     .merge(df_audit, on='application_id')
)
df.to_csv('data/os_audit/results.csv')